<a href="https://colab.research.google.com/github/Micahgs/Data-science-Assignments/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score




In [32]:
df= pd.read_csv("https://raw.githubusercontent.com/Micahgs/DATA-science-asiignment-datasets/refs/heads/main/anime.csv")

In [33]:
#Basic Exploration
df.info()
df.isnull().sum()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# Data Cleanning

In [34]:
df = df.dropna(subset=["genre", "rating"]).copy()  # Remove rows with missing genre or rating
df.loc[:, "episodes"] = df["episodes"].replace("Unknown", np.nan)
df = df.dropna(subset=["episodes"])
df["episodes"] = df["episodes"].astype(int)

# Encoding

In [35]:
df["genre"] = df["genre"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)
genre_exploded = df["genre"].explode()
genre_dummies = pd.get_dummies(genre_exploded).groupby(level=0).sum()

# Normalize numerical features

In [36]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[["rating", "members", "episodes"]])
scaled_df = pd.DataFrame(scaled, columns=["rating", "members", "episodes"], index=df.index)


In [37]:
features = pd.concat([scaled_df, genre_dummies], axis=1)

# Cosine Similarity

In [38]:
cos_sim_matrix = cosine_similarity(features)

# Recommendation Function

In [39]:
def recommend_anime(title, top_n=10, threshold=0.7):
    if title not in df["name"].values:
        return f"'{title}' not found in dataset."

    idx = df[df["name"] == title].index[0]
    sim_scores = list(enumerate(cos_sim_matrix[idx]))

    # Filter by threshold
    sim_scores = [(i, score) for i, score in sim_scores if i != idx and score >= threshold]

    # Sort by similarity and pick top_n
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:top_n]
    indices = [i for i, _ in sim_scores]

    return df.iloc[indices][["name", "genre", "type", "rating"]]


# Experement with diffrent thersholds

In [46]:
def recommend_anime(title, top_n=10, threshold=0.6):
    if title not in df["name"].values:
        return f"'{title}' not found in dataset."

    idx = df[df["name"] == title].index[0]
    sim_scores = list(enumerate(cos_sim_matrix[idx]))

    # Filter by threshold
    sim_scores = [(i, score) for i, score in sim_scores if i != idx and score >= threshold]

    # Sort by similarity and pick top_n
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:top_n]
    indices = [i for i, _ in sim_scores]

    return df.iloc[indices][["name", "genre", "type", "rating"]]


In [40]:
recommend_anime("Naruto", threshold=0.85)

,name,genre,type,rating
4540,Trava: Fist Planet,"[Action, Comedy, Mecha, Sci-Fi, Space]",OVA,6.70
1906,Mobile Suit Gundam 00 The Movie: A Wakening of...,"[Action, Mecha, Sci-Fi, Space]",Movie,7.41
2130,Super Robot Taisen OG: The Inspector,"[Action, Mecha, Sci-Fi, Space]",TV,7.35
2165,Sei Juushi Bismarck,"[Action, Mecha, Sci-Fi, Space]",TV,7.34
3182,Rinne no Lagrange Season 2,"[Action, Comedy, Mecha, Sci-Fi]",TV,7.06
3345,Rinne no Lagrange,"[Action, Comedy, Mecha, Sci-Fi]",TV,7.02
4601,iDOLM@STER Xenoglossia,"[Action, Comedy, Mecha, Sci-Fi]",TV,6.68
2828,Aldnoah.Zero 2nd Season,"[Action, Mecha, Sci-Fi, Space]",TV,7.15
4749,Uchuu Senshi Baldios (Movie),"[Action, Mecha, Sci-Fi, Space]",Movie,6.65
4776,Muteki Robo Trider G7,"[Comedy, Mecha, Sci-Fi, Space]",TV,6.64


In [41]:
recommend_anime("Naruto", threshold=0.8)

,name,genre,type,rating
4540,Trava: Fist Planet,"[Action, Comedy, Mecha, Sci-Fi, Space]",OVA,6.70
1906,Mobile Suit Gundam 00 The Movie: A Wakening of...,"[Action, Mecha, Sci-Fi, Space]",Movie,7.41
2130,Super Robot Taisen OG: The Inspector,"[Action, Mecha, Sci-Fi, Space]",TV,7.35
2165,Sei Juushi Bismarck,"[Action, Mecha, Sci-Fi, Space]",TV,7.34
3182,Rinne no Lagrange Season 2,"[Action, Comedy, Mecha, Sci-Fi]",TV,7.06
3345,Rinne no Lagrange,"[Action, Comedy, Mecha, Sci-Fi]",TV,7.02
4601,iDOLM@STER Xenoglossia,"[Action, Comedy, Mecha, Sci-Fi]",TV,6.68
2828,Aldnoah.Zero 2nd Season,"[Action, Mecha, Sci-Fi, Space]",TV,7.15
4749,Uchuu Senshi Baldios (Movie),"[Action, Mecha, Sci-Fi, Space]",Movie,6.65
4776,Muteki Robo Trider G7,"[Comedy, Mecha, Sci-Fi, Space]",TV,6.64


In [42]:
recommend_anime("Naruto", threshold=0.95)

,name,genre,type,rating
4540,Trava: Fist Planet,"[Action, Comedy, Mecha, Sci-Fi, Space]",OVA,6.7


# Evaluation:

In [49]:
ground_truth = {
    "Naruto": [
        "Naruto: Shippuuden Movie 1",
        "Naruto: Shippuuden Movie 2 - Kizuna",
        "Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono",
        "Naruto: Shippuuden Movie 4 - The Lost Tower",
        "Naruto: Shippuuden Movie 5 - Blood Prison"
    ],
    "Death Note": [
        "Code Geass: Hangyaku no Lelouch",
        "Code Geass: Hangyaku no Lelouch R2",
        "Zankyou no Terror",
        "Monster"
    ],
    "Fullmetal Alchemist: Brotherhood": [
        "Steins;Gate",
        "Hunter x Hunter (2011)",
        "Attack on Titan",
        "Code Geass: Hangyaku no Lelouch"
    ]
}

# Evaluation function
def evaluate_recommendation_system(top_n=10, threshold=0.7):
    y_true = []
    y_pred = []

    print("--- Evaluation Results ---\n")

    for title, true_list in ground_truth.items():
        print(f"Evaluating: {title}")

        result = recommend_anime(title, top_n=top_n, threshold=threshold)
        if isinstance(result, str):
            print(f"  {result}")
            continue

        predicted_list = result["name"].tolist()
        true_set = set(true_list)
        predicted_set = set(predicted_list)

        for anime in true_set:
            y_true.append(1)
            y_pred.append(1 if anime in predicted_set else 0)

        # Display of sample result
        print(f"  Predicted: {predicted_list[:3]}...")
        print(f"  Ground Truth Matches: {len(true_set & predicted_set)} of {len(true_set)}\n")

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(" Overall Evaluation Metrics:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall:    {recall:.2f}")
    print(f"  F1 Score:  {f1:.2f}")


In [48]:
evaluate_recommendation_system(top_n=10, threshold=0.7)


--- Evaluation Results ---

Evaluating: Naruto
  Predicted: ['Trava: Fist Planet', 'Mobile Suit Gundam 00 The Movie: A Wakening of the Trailblazer', 'Super Robot Taisen OG: The Inspector']...
  Ground Truth Matches: 0 of 5

Evaluating: Death Note
  Predicted: ['Death Note Rewrite', 'Higurashi no Naku Koro ni Kai', 'Mousou Dairinin']...
  Ground Truth Matches: 0 of 4

Evaluating: Fullmetal Alchemist: Brotherhood
  Predicted: ['Fullmetal Alchemist', 'Fullmetal Alchemist: The Sacred Star of Milos', 'Fullmetal Alchemist: Brotherhood Specials']...
  Ground Truth Matches: 0 of 4

 Overall Evaluation Metrics:
  Precision: 0.00
  Recall:    0.00
  F1 Score:  0.00


# Interview Questions:

1. Can you explain the difference between user-based and item-based collaborative filtering?

Yes. Both are techniques used in recommendation systems, but they focus on different things.

User-based collaborative filtering tries to find users who are similar to you, based on their past behavior, like what they’ve liked or rated. Then it recommends items that those similar users liked but you haven’t seen yet. It’s like saying, "People like you also liked this."

Item-based collaborative filtering works the other way. It looks at items (like movies or anime) that are similar to each other based on how users have rated them. Then it recommends items similar to what you’ve liked before. It’s more like saying, "Because you liked this anime, here are some similar ones."

2. What is collaborative filtering, and how does it work?

Collaborative filtering is a recommendation technique that makes suggestions by learning from user behavior — not from the content of the items.

Instead of looking at what the anime is about, it looks at how people interact with it. For example, if two users have watched and rated many of the same shows, the system assumes they have similar tastes. So, if one of them watches a new anime, the other might get that as a recommendation.

There are two main types: user-based and item-based. Both use the idea that patterns in user behavior can help predict what someone might like next.

